In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import scipy as sc
import statsmodels.api as sm
import scipy.stats as stats
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy as cp
import stormeunice as eun
import multiprocessing
import glob
import dask

sns.set_theme(style="white")
sns.set_style("white")
%matplotlib inline

random.seed(10)

In [2]:
def lagrangian_frame(ds):
    ds = ds.squeeze()
    ds = ds.assign_coords(latitude=ds.latitude-ds.centroid_lat,longitude=ds.longitude-ds.centroid_lon)
    ds = ds.rename(latitude='storm_lat',longitude='storm_lon')
    ds = ds.sel(storm_lon=slice(-10,10),storm_lat=slice(10,-10))
    return ds

In [3]:
def import_medr_tracks_TE(fpath):
    
    df = pd.read_csv(fpath,skipinitialspace=True)
    
    expdict = {'1':'ENS','b2nn':'pi','b2nq':'pi','b2ns':'pi','b2no':'incr','b2nr':'incr','b2nt':'incr'}
    
    fname = fpath.split('/')[-1]
    _,expid,inidate,mem = fname.split('_')
    
    df['expid'] = expid
    df['experiment'] = expdict[expid]
    df['inidate'] = pd.to_datetime(inidate)
    df['number'] = int(mem)
    
    return df

In [4]:
P1 = multiprocessing.Pool(40)

ifs_tracks = P1.map(import_medr_tracks_TE, glob.glob('/home/l/leach/Attribution/NA-Storms/Nick/Scripts/tracking/TE/TEStitch_[!2]*'))

P1.close()

ifs_tracks = pd.concat(ifs_tracks)
ifs_tracks['date'] = pd.to_datetime(ifs_tracks.loc[:,['year','month','day','hour']])

## add era track in
era_track = pd.read_csv('/home/l/leach/Attribution/NA-Storms/Nick/Scripts/tracking/TE/TEStitch_2022_0',skipinitialspace=True)

era_track['expid'] = 'era5'
era_track['experiment'] = 'era5'
era_track['inidate'] = pd.to_datetime('2022-01-01')
era_track['number'] = 0
era_track['date'] = pd.to_datetime(era_track.loc[:,['year','month','day','hour']])

all_tracks = pd.concat([ifs_tracks,era_track])

eunice_track = era_track.query('track_id==5')

In [5]:
ifs_tracks_filter = ifs_tracks.groupby(['track_id','inidate','experiment','number']).agg(dict(date=min, day=np.size, lon=lambda x: x.iloc[0], lat=lambda x: x.iloc[0], msl=min))
eunice_filter = eunice_track.groupby(['track_id','inidate','experiment','number']).agg(dict(date=min, day=np.size, lon=lambda x: x.iloc[0], lat=lambda x: x.iloc[0], msl=min)).iloc[0]

# starttime criterion - must be detected within 12 hours (before or after) the ERA5 track detectino time
starttime_crit = (ifs_tracks_filter.date - eunice_filter.date).dt.total_seconds().abs()<(12*60*60)
# starting location criterion - must start within 10 degrees of the ERA5 track starting point
startloc_crit = np.sqrt((ifs_tracks_filter.lon-eunice_filter.lon)**2+(ifs_tracks_filter.lat-eunice_filter.lat)**2)<10
# depth criterion - must reach a depth of at least 980 hPa
minmsl_crit = ifs_tracks_filter.msl < 98000

ifs_tracks_filtered = ifs_tracks_filter.loc[starttime_crit&startloc_crit&minmsl_crit]
ifs_eunice_list = ifs_tracks.set_index(['track_id','inidate','experiment','number']).sort_index().loc[ifs_tracks_filtered.index].reset_index()

In [13]:
def preproc_to_stormframe(ds):
    
    ds = ds.copy()
    
    if 'number' not in ds.coords:
        ds = ds.expand_dims({'number':[0]})
        
    fpath = ds.encoding['source']
    
    exp = fpath.split('/')[-5]
    inidate = fpath.split('/')[-1].split('_')[-1].split('.')[0]
    
    ds_tracks = ifs_eunice_list.query('experiment=="{}" & inidate=="{}"'.format(exp,inidate))

    LG_fields = []

    for num in set(ds.number.values).intersection(ds_tracks.number.unique()):

        mem_track = ds_tracks.loc[ds_tracks.number==num]
        mem_fields = ds.sel(number=num)
        time_intersection = sorted(list(set(mem_fields.time.values).intersection(mem_track.date.values)))
        
        resample_freq = 3 ## resampling frequency in hours
        if inidate == '2022-02-10':
            resample_freq = 6
        
        ## get start / end times for properly calculating the maximum fields (taking into account the different preproc times in IFS)
        time_start = time_intersection[0]-pd.Timedelta('{}h 59m'.format(resample_freq-1))
        time_end = time_intersection[-1]

        ## get the instantaneous fields + wind speeds
        mem_fields_out = mem_fields.get(['sst','u10','v10','msl','u100','v100','tcwv']).sel(time = time_intersection)
        mem_fields_out['ws10'] = np.sqrt(mem_fields_out.u10**2+mem_fields_out.v10**2)
        mem_fields_out['ws100'] = np.sqrt(mem_fields_out.u100**2+mem_fields_out.v100**2)
        
        ## get the maximum fields, taking into account the different preproc times
        mxtpr_field_out = mem_fields.mxtpr.sel(time=slice(time_start, time_end)).resample(time='{}h'.format(resample_freq), label='right', closed='right', base=0).max()
        mem_fields_out['mxtpr'] = mxtpr_field_out

        ## add in the mslp centroid lon/lats for Lagrangian analysis 
        mem_track_out = mem_track.loc[mem_track.date.isin(time_intersection)]
        mem_fields_out['centroid_lon'] = ('time',(mem_track_out.lon*4).round()/4)
        mem_fields_out['centroid_lat'] = ('time',(mem_track_out.lat*4).round()/4)

        ## convert to storm frame fields
        mem_fields_out = mem_fields_out.groupby('time').apply(lagrangian_frame)
        mem_fields_out = mem_fields_out.assign(datetime=mem_fields_out.time).drop('time').rename(time='timestep')
        
        ## compute the time of peak vorticity (include moving average to smooth) for storm composites
        peak_vo = mem_track.rolling(3,center=True).mean().vo.idxmax()
        peak_vo_datetime = mem_track.date.loc[peak_vo]
        peak_vo_relative_time = (mem_fields_out.datetime.squeeze().to_pandas() - peak_vo_datetime).dt.total_seconds().values/(3600*24)

        ## set the storm frame fields timestep relative to peak vorticity time
        mem_fields_out = mem_fields_out.assign_coords(timestep=peak_vo_relative_time)

        LG_fields += [mem_fields_out]
        
    LG_fields = xr.concat(LG_fields,'number')
    
    LG_fields = LG_fields.expand_dims(dict(inidate=[pd.to_datetime(inidate)],experiment=[exp]))
    
    return LG_fields

In [14]:
ens_storms = xr.open_mfdataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/pf/*.nc', preprocess=preproc_to_stormframe, combine='nested',concat_dim=['inidate'],parallel=True, engine='netcdf4')
# pi_storms = xr.open_mfdataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/pi/EU025/sfc/pf/*.nc',preprocess=preproc_to_stormframe,combine='nested',concat_dim=['inidate'],parallel=True, engine='netcdf4')
# incr_storms = xr.open_mfdataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/incr/EU025/sfc/pf/*.nc',preprocess=preproc_to_stormframe,combine='nested',concat_dim=['inidate'],parallel=True, engine='netcdf4')

/tmp/user/27529/ipykernel_3482090/3636006927.py:50: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['date', 'experiment', 'inidate'], dtype='object')
  peak_vo = mem_track.rolling(3,center=True).mean().vo.idxmax()
/tmp/user/27529/ipykernel_3482090/3636006927.py:50: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['date', 'experiment', 'inidate'], dtype='object')
  peak_vo = mem_track.rolling(3,center=True).mean().vo.idxmax()
/tmp/user/27529/ipykernel_3482090/3636006927.py:50: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped 

ValueError: conflicting sizes for dimension 'time': length 17 on 'centroid_lon' and length 10 on {'time': 'sst', 'latitude': 'sst', 'longitude': 'sst'}

/tmp/user/27529/ipykernel_3482090/3636006927.py:50: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['date', 'experiment', 'inidate'], dtype='object')
  peak_vo = mem_track.rolling(3,center=True).mean().vo.idxmax()
/tmp/user/27529/ipykernel_3482090/3636006927.py:50: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['date', 'experiment', 'inidate'], dtype='object')
  peak_vo = mem_track.rolling(3,center=True).mean().vo.idxmax()
/tmp/user/27529/ipykernel_3482090/3636006927.py:50: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped 

In [23]:
ens_storms.sel(number=1, time='2022-02-18 12')

<xarray.Dataset>
Dimensions:    (longitude: 289, latitude: 163, inidate: 3)
Coordinates:
  * longitude  (longitude) float32 -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * latitude   (latitude) float32 73.5 73.25 73.0 72.75 ... 33.5 33.25 33.0
    number     int32 1
    time       datetime64[ns] 2022-02-18T12:00:00
Dimensions without coordinates: inidate
Data variables: (12/14)
    sst        (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    fg10       (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    tcw        (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    tcwv       (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    msl        (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    tcc        (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    ...         ...
    u10        (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    v10        (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    u100       (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    v100       (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    mxcape6    (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
    mxcapes6   (inidate, latitude, longitude) float32 dask.array<chunksize=(1, 163, 289), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-12-08 19:41:25 GMT by grib_to_netcdf-2.26.0: grib_to_n...